In [8]:
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re
import requests
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

Product_name,Prices,Description,Ratings,PLink,CLink,Iurl=[],[],[],[],[],[],[]
name,description,price,rating,plink,iurl=None,None,None,None,None,None
Images,mainName,sellerName,sellerRating,Specification,productPrices,productReviews,Size,CompleteLink = [],[],[],[],[],[],[],[],[]
images,name,sellerN,sellerR,infos,infoDetails,productPrice,stars,totreviews=None,None,None,None,None,None,None,None,None

class Flipkart:
    def __init__(self, driver=r"C:\Users\HP\Downloads\WEB SCRAPING\chromedriver.exe"):
        self.driver=driver

    def scraping(self, req_url):
        self.driver.get(req_url)
        #time.sleep(1)
        html = self.driver.page_source
        return html
    
    def clicking(self, req_url):
        self.driver.get(req_url)
        time.sleep(1)
        try:
            clicker_element = self.driver.find_element(By.CSS_SELECTOR, 'div._1qs4Jt')
            clicker_element.click()
            time.sleep(1)
        except NoSuchElementException:
            pass
        html = self.driver.page_source
        return html
    
    def scrape_url(self,url):            
        flag=0
        pattern = r"https:\/\/([\w.-]+)\/"
        match=re.search(pattern,url)
        extract=match.group(1)

        r=requests.get(url)
        htmlContent=r.content
        soup=BeautifulSoup(htmlContent,'html.parser')
        pages=soup.find('div',class_='_2MImiq')

        if pages is not None:
            pages=pages.find('span').text
        else:
            pages=str(1)
        totalpages=int(pages.split()[-1].replace(',',''))

        for i in range(1,totalpages+1):
            htmlContent=self.scraping(url+'&page='+str(i))
            time.sleep(1)
            soup=BeautifulSoup(htmlContent,'html.parser')
            boxes=soup.findAll('div')
            for box in boxes:
                if '_2kHMtA' in str(box):
                    boxes=soup.findAll('div', class_='_2kHMtA')
                    flag=0
                    break
                elif '_4ddWXP' in str(box):
                    boxes=soup.findAll('div', class_='_4ddWXP')
                    flag=1
                    break
                elif '_1xHGtK _373qXS' in str(box):
                    boxes=soup.findAll('div',class_='_1xHGtK _373qXS')
                    flag=2
                    break
            for box in boxes:
                if (flag==0):
                    name = box.find('div', class_='_4rR01T')
                    if name is not None:
                        name = name.text
                    else:
                        name = np.nan        
                    price = box.find('div', class_='_30jeq3 _1_WHN1')
                    if price is not None:
                        price = price.text
                    else:
                        price = np.nan
                    rating = box.find('div', class_='_3LWZlK')
                    if rating is not None:
                        rating = rating.text
                    else:
                        rating = np.nan
                    plink = box.find('a', class_='_1fQZEK')
                    if plink is not None:
                        plink = plink.get('href')
                    else:
                        plink = np.nan
                    iurl = box.find('img', class_='_396cs4')
                    if iurl is not None:
                        iurl = iurl.get('src')
                    else:
                        iurl = np.nan
                elif(flag==1):
                    name = box.find('a', class_='s1Q9rs')
                    if name is not None:
                        name = name.get('title')
                    else:
                        name = np.nan
                    price = box.find('div', class_='_30jeq3')
                    if price is not None:
                        price=price.text
                    rating = box.find('div', class_='_3LWZlK')
                    if rating is not None:
                        rating = rating.text
                    else:
                        rating = np.nan
                    plink = box.find('a', class_='_8VNy32')
                    if plink is not None:
                        plink = plink.get('href')
                    else:
                        plink = np.nan
                    iurl = box.find('img', class_='_396cs4')
                    if iurl is not None:
                        iurl = iurl.get('src')
                    else:
                        iurl = np.nan
                elif(flag==2):
                    rating=np.nan
                    name = box.find('a', class_='IRpwTa')
                    if name is not None:
                        name=name.get('title')
                    else:
                        name=np.nan
                    price = box.find('div', class_='_30jeq3')
                    if price is not None:
                        price=price.text
                    else:
                        price=np.nan
                    plink = box.find('a', class_='_3bPFwb')
                    if plink is not None:
                        plink = plink.get('href')
                    else:
                        plink = np.nan
                    iurl = box.find('img', class_='_2r_T1I')
                    if iurl is not None:
                        iurl = iurl.get('src')
                    else:
                        iurl = np.nan

                Product_name.append(name)
                Prices.append(price)
                Ratings.append(rating)
                PLink.append(plink)
                CLink.append(extract+plink)
                Iurl.append(iurl)
        #flipkart.driver.quit()
        df=pd.DataFrame({"Product Name":Product_name,"Prices":Prices,"Rating":Ratings,"Partial Link":PLink,"Complete Link":CLink,"Image Url":Iurl})
        return(df)

    def scrape_each_page(self): 
        flag_for_rating=0
        size_exists=0
        for i in CLink:
            htmlContent = self.clicking('http://'+i)
            time.sleep(1)
            soup = BeautifulSoup(htmlContent, 'html.parser')
            box = soup.find('div', class_='_1YokD2 _2GoDe3')
            images = box.find_all('img', class_='q6DClP')
            name=box.find('span', class_='B_NuCI')
            sellerN=box.find('div',class_='_1RLviY')
            sellerR=box.find('div',class_='_3LWZlK _1D-8OL')

            if box.find('div',class_='_3dtsli') is not None:
                infos=box.findAll('td',class_='_1hKmbr col col-3-12')
                infoDetails=box.findAll('td',class_='URwL2w col col-9-12')
            elif box.find('div',class_='_2yIA0Y') is not None:
                infos=box.findAll('div',class_='col col-3-12 _2H87wv')
                infoDetails=box.findAll('div',class_='col col-9-12 _2vZqPX')

            productPrice=box.find('div',class_='_30jeq3 _16Jk6d')

            if box.find('div',class_='col JOpGWq') is not None:
                stars=box.findAll('span',class_='_26f_zl')
                totreviews=box.findAll('div',class_='_1uJVNT')
                flag_for_rating=0
            elif box.find('div',class_='col JOpGWq _33R3aa') is not None:
                star=box.find('div',class_='_3LWZlK _138NNC')
                flag_for_rating=1

            imagelist=[]
            for image in images:
                if image is not None:
                    image=image.get('src')
                    imagelist.append(image)
                else:
                    imagelist.append(np.nan)
            if name is not None:
                    name=name.text
            else:
                    name=np.nan
            if sellerN is not None:
                    sellerN=sellerN.text
            else:
                    sellerN=np.nan
            if sellerR is not None:
                    sellerR=sellerR.text
            else:
                    sellerR=np.nan
            if productPrice is not None:
                    productPrice=productPrice.text
            else:
                    productPrice=np.nan


            product_info,product_review={},{}
            for info, infoDetail in zip(infos, infoDetails):
                if info is not None and infoDetail is not None:
                    info=info.text
                    infoDetail=infoDetail.text
                    product_info[info]=infoDetail
                else:
                    info=np.nan
                    infoDetail=np.nan
                    product_info[info]=infoDetail

            if flag_for_rating==0:
                for star,totreview in zip(stars,totreviews):
                    if star is not None and totreview is not None:
                        star=star.text
                        totreview=totreview.text
                        product_review[star]=totreview
                    else:
                        star=np.nan
                        totreview=np.nan
                        product_review[star]=totreview
            elif flag_for_rating==1:
                if star is not None:
                    product_review=star.text
                else:
                    product_review=np.nan

            size_chart_list=[]
            chart = soup.find('div',class_='_1qs4Jt')
            if (chart != None):
                size_exists=1
                tds = soup.findAll('td', class_='eKQwAg')
                count = len(list(tds))
                eachs=soup.findAll('td',class_='_2kfRkH')
                k=0
                size_chart={}
                if (k!=count):
                    for td in tds:
                        innerlist=[]
                        td=td.text
                        for j in range(0, len(eachs), count):
                            innerlist.append(eachs[k+j].text)
                        size_chart[td]=innerlist
                        k=k+1
                    size_chart_list.append(size_chart)
            else:
                size_chart_list.append(np.nan)

            Images.append(imagelist)
            mainName.append(name)
            sellerName.append(str(sellerN).replace(str(sellerR),''))
            sellerRating.append(sellerR)
            Specification.append(product_info)
            productPrices.append(productPrice)
            productReviews.append(product_review)
            Size.append(size_chart_list)
            CompleteLink.append(i)

        if (size_exists==0):
            df=pd.DataFrame({"Product Name": mainName,"Price": productPrices,"Image List": Images,"Seller Name":sellerName,"Seller Rating": sellerRating,"Specifications": Specification,"Review": productReviews,"Complete Link":CompleteLink})
        else:
            df=pd.DataFrame({"Product Name": mainName,"Price": productPrices,"Image List": Images,"Seller Name":sellerName,"Seller Rating": sellerRating,"Specifications": Specification,"Review": productReviews,"Size Chart": Size,"Complete Link":CompleteLink})
        return(df)

    def merge(self, dataframe1, dataframe2):
        merged_df=pd.merge(dataframe1, dataframe2,on=['Complete Link'])
        return merged_df

    def control(self,url):
        self.driver=webdriver.Chrome(self.driver)
        dataframe1=self.scrape_url(url)
        dataframe2=self.scrape_each_page()
        merged_df=self.merge(dataframe1,dataframe2)
        self.driver.quit()
        return merged_df
        
url='https://www.flipkart.com/search?q=dunder+mifflin+t+shirt&otracker=AS_Query_HistoryAutoSuggest_5_0&otracker1=AS_Query_HistoryAutoSuggest_5_0&marketplace=FLIPKART&as-show=on&as=off&as-pos=5&as-type=HISTORY'
flipkart=Flipkart()
merged_df=flipkart.control(url)
merged_df

C:\Users\HP\AppData\Local\Temp\ipykernel_4060\3420071268.py:281: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver=webdriver.Chrome(self.driver)


,Product Name_x,Prices,Rating,Partial Link,Complete Link,Image Url,Product Name_y,Price,Image List,Seller Name,Seller Rating,Specifications,Review,Size Chart
0,Solid Men Round Neck Poly Cotton Dark Green T-...,₹299,NaN,/metronaut-solid-men-round-neck-dark-green-t-s...,www.flipkart.com/metronaut-solid-men-round-nec...,https://rukminim1.flixcart.com/image/612/612/x...,Solid Men Round Neck Poly Cotton Dark Green T-...,₹299,[https://rukminim1.flixcart.com/image/128/128/...,THEEGSTORE,3.6,"{'Type': 'Round Neck', 'Sleeve': 'Short Sleeve...",3.9,"[{'Size': ['XS', 'S', 'M', 'L', 'XL', 'XXL'], ..."
1,Men Graphic Print Round Neck Cotton Blend Blac...,₹269,NaN,/bglore-graphic-print-men-round-neck-black-t-s...,www.flipkart.com/bglore-graphic-print-men-roun...,https://rukminim1.flixcart.com/image/612/612/x...,Men Graphic Print Round Neck Cotton Blend Blac...,₹269,[https://rukminim1.flixcart.com/image/128/128/...,HIGHSTREET TRENDZ LLP,4.1,"{'Type': 'Round Neck', 'Sleeve': 'Half Sleeve'...",3.6,"[{'Size': ['M', 'L', 'XL'], 'Chest': ['38', '4..."
2,The Office Dunder Mifflin Men Printed Round Ne...,₹419,NaN,/krazy-kameez-printed-men-round-neck-black-t-s...,www.flipkart.com/krazy-kameez-printed-men-roun...,https://rukminim1.flixcart.com/image/612/612/k...,The Office Dunder Mifflin Men Printed Round Ne...,₹419,[https://rukminim1.flixcart.com/image/128/128/...,EunoiaGlobal,3.7,"{'Type': 'Round Neck', 'Sleeve': 'Short Sleeve...",3.1,"[{'Size': ['XS', 'S', 'M', 'L', 'XL', 'XXL'], ..."


In [ ]:
#JUST4U02223